## Embedding model ##
>Floris Menninga \
>Datum: 8-01-2026

In [1]:
# %pip install tensorflow keras
# #%pip install tqdm
# %pip install tensorflow_text
# %pip install bs4
# %pip install joblib
# %pip install lxml

from data_parser import xml_parser
import EmbeddingModel

import io
import re
import string
import tqdm
import os

import numpy as np
import requests
import tensorflow as tf
import tensorflow_text as tf_text
import joblib
import tensorflow as tf
from tensorflow.keras import layers
import lxml

%load_ext tensorboard
AUTOTUNE = tf.data.AUTOTUNE
SEED = 0

2026-01-24 19:46:38.535114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769280398.643686   35445 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769280398.677818   35445 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769280398.852898   35445 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769280398.852928   35445 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769280398.852932   35445 computation_placer.cc:177] computation placer alr

In [2]:
os.environ['LD_LIBRARY_PATH'] = '/run/opengl-driver/lib:' + os.environ.get('LD_LIBRARY_PATH', '')
print("Gpu's: ", len(tf.config.list_physical_devices('GPU')))

Gpu's:  0


2026-01-24 19:46:44.925603: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2026-01-24 19:46:44.925770: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2026-01-24 19:46:44.925782: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: FLORIS-2
2026-01-24 19:46:44.925788: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] hostname: FLORIS-2
2026-01-24 19:46:44.925869: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:197] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program. The library may be missing or provided via another object.
2026-01-24 19:46:44.926010: I exte

## Tokenizer

De eerste stap is het omzetten van de text in tokens:
In tegenstelling to methodes zoals bytepair encoding etc. worden de tokens gemaakt door de zinnen \
te splitten op spaties. 

In [84]:
sentence = "Hallo, dit is een testzin om het programma te testen."
tokens = list(sentence.lower().split())
print(len(tokens))

10


### Vocabulary maken van deze tokens:

In [85]:
vocab, index = {}, 1 
vocab['<pad>'] = 0
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)

print(vocab)

{'<pad>': 0, 'hallo,': 1, 'dit': 2, 'is': 3, 'een': 4, 'testzin': 5, 'om': 6, 'het': 7, 'programma': 8, 'te': 9, 'testen.': 10}


### Inverse vocabulary:
Van integer index naar token:

In [86]:
inverse_vocab = {index: token for token, index in vocab.items()}

print(inverse_vocab)

{0: '<pad>', 1: 'hallo,', 2: 'dit', 3: 'is', 4: 'een', 5: 'testzin', 6: 'om', 7: 'het', 8: 'programma', 9: 'te', 10: 'testen.'}


## Vectorizeren van de zin:

In [87]:
test_sequence = [vocab[word] for word in tokens]

print(test_sequence)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Skip-gram maken:
Het embedding model zal gemaakt worden met behulp van het Word2Vec algorithme. Verdere uitleg over hoe dit algorithme werkt komt straks maar om dit te kunnen gebruiken 
moet er eerst een Skip-gram model gemaatk worden. 
Het belangrijkste punt van een skip-gram is dat de vector representatie gebaseerd is op de ruimtelijke nabijheid van woorden in een zin. 
Als het woord "Ketting" altijd gevolgd zou worden door het woord "zaag" zal deze combinatie van woorden vaker voorkomen dan in andere combinaties.
Een skip-gram model is een klein neural network met een inputlayer, embedding layer en output layer. 
Dit model moet een waarschijnlijkheids verdelings vector geven voor een gegeven input woord. Dat is dus de kans dat deze twee woorden samen voorkomen binnen de context lengte waarmee het model getrained is. De som van deze waarschijnlijkheids verdeling is 1.

In [88]:
window_size = 2

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      test_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0,
      seed=0
)

print(len(positive_skip_grams))

34


In [89]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")


(3, 4): (is, een)
(1, 2): (hallo,, dit)
(8, 9): (programma, te)
(6, 8): (om, programma)
(4, 3): (een, is)


In [90]:
target_word, context_word = positive_skip_grams[0]

num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])


tf.Tensor([9 3 0 2], shape=(4,), dtype=int64)
['te', 'is', '<pad>', 'dit']


In [91]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word


In [92]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")


target_index    : 3
target_word     : is
context_indices : [4 9 3 0 2]
context_words   : ['een', 'te', 'is', '<pad>', 'dit']
label           : [1 0 0 0 0]


In [93]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)


[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=4,
          seed=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=0,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels



## Word2Vec:

Het word2vec algorithme is een word embedding techniek in natural language processing die er voor zorgt dat woorden als vectors gerepresenteerd kunnen worden in een continue vector ruimte. 
Dit kan vervolgens gebruikt worden om relaties tussen woorden te achterhalen.



Hier zal de dataset ingeladen worden:

In [95]:
#text_ds = tf.data.TextLineDataset(["/run/media/floris/FLORIS_3/Data_set/PubMed/dataset_100914.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))
text_ds = tf.data.TextLineDataset(["/home/floris/Documenten/Dataset/trainingsData.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [96]:
# Verwijder punctuatie / hoofdletters.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Aantal woorden in seq en vocabulary grootte:
sequence_length = 1000
vocab_size = 1000

# Vectorize the layer en split en map strings tokens met TextVectorization:
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


In [97]:
vectorize_layer.adapt(text_ds.batch(512))

In [98]:
# Inverse vocab:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', np.str_('the'), np.str_('of'), np.str_('and'), np.str_('in'), np.str_('to'), np.str_('a'), np.str_('for'), np.str_('with'), np.str_('were'), np.str_('was'), np.str_('that'), np.str_('is'), np.str_('cancer'), np.str_('by'), np.str_('as'), np.str_('cells'), np.str_('or'), np.str_('from')]


In [99]:
# Vectoriseer de data in text_ds:
text_vector_ds = text_ds.batch(512).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [100]:
# Maak een lijst van deze gevectoriseerde data:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

219


In [101]:
# for seq in sequences[:5]:
#   print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

In [102]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=5,
    num_ns=4,
    vocab_size=vocab_size,
    seed=0)


# print(f"targets.shape: {targets.shape}")
# print(f"contexts.shape: {contexts.shape}")
# print(f"labels.shape: {labels.shape}")

100%|██████████| 219/219 [04:59<00:00,  1.37s/it]


Gegenereerde data opslaan:
Het duurde lang om te genereren dus sla ik het op in een .joblib bestand, dit zal ik ook voor sommige vervolg stappen doen.


In [103]:
# joblib_list = [targets, contexts, labels]

# joblib.dump(joblib_list, "targets_contexts_labels.joblib")

In [104]:
BATCH_SIZE = 20
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(20,), dtype=tf.int32, name=None), TensorSpec(shape=(20, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(20, 5), dtype=tf.int64, name=None))>


In [105]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [106]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

Hier wordt het Word2Vec algorithme gebruikt 

In [107]:
embedding_dim = 64
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [108]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [109]:
word2vec.fit(dataset, epochs=50, callbacks=[tensorboard_callback])

Epoch 1/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 40s 1ms/step - accuracy: 0.5450 - loss: 1.1491
Epoch 2/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 37s 1ms/step - accuracy: 0.5578 - loss: 1.1176
Epoch 3/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 36s 1ms/step - accuracy: 0.5737 - loss: 1.0847
Epoch 4/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 39s 1ms/step - accuracy: 0.5874 - loss: 1.0569
Epoch 5/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step - accuracy: 0.5961 - loss: 1.0410
Epoch 6/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 36s 1ms/step - accuracy: 0.6011 - loss: 1.0323
Epoch 7/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 37s 1ms/step - accuracy: 0.6043 - loss: 1.0270
Epoch 8/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.6067 - loss: 1.0234
Epoch 9/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 39s 1ms/step - accuracy: 0.6086 - loss: 1.0207
Epoch 10/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.6100 - loss: 1.0187
Epoch 11/50
26338/26338 ━━━━━━━━━━━━━━━━━━━━ 39s 1ms/step - accuracy: 0.6113 - loss: 1.01

In [110]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [111]:
num_tokens = min(len(vocab), len(weights))

with io.open('vectors.tsv', 'w', encoding='utf-8') as out_v, \
     io.open('metadata.tsv', 'w', encoding='utf-8') as out_m:

    # Skip de padding token index 0:
    for index in range(1, num_tokens):
        word = vocab[index]
        vec = weights[index]
        
        clean_word = word.strip()
        
        if not clean_word:
            clean_word = "[UNK_EMPTY]"

        # Vector
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        
        # Metadata
        out_m.write(clean_word + "\n")

print(f"{num_tokens - 1} vectors en metadata entries.")

999 vectors en metadata entries.


## Trainen van het model:

In [112]:
# model = EmbeddingModel.EmbeddingModel()
# model.load_dataset()
# model.create_vocab()
# model.vectorize()
# model.generate_trainingdata()
# model.word2vec()
# model.fit()
# model.save2file()

## Data:
Als trainingsdata gebruik ik 869597 Pubmed artikelen (~70GB) die ik gedownload heb van https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/

oa_comm_xml.PMC000xxxxxx.baseline.2025-06-26.tar.gz t/m oa_comm_xml.PMC004xxxxxx.baseline.2025-06-26.tar.gz

Deze zijn nieuwer dan de artikelen uit 2021 die op /commons/data/NCBI/PubMed/ staan.

Vervolgens moeten deze artikelen doorzocht worden op inhoud. \
De .xml bestanden hebben headers voor titel, abstract, body etc. deze doorzoek ik met behulp van mijn klasse: data_parser.py

Het filteren op de drie keywords "cancer", "melanoma" en "carcinoma" duurde 9 uur en 30 minuten en reduceerde de hoeveelheid artikelen 
van 869597 naar 100914 (2.7GB).

100914 Artikelen met deze zoekcriteria opgeslagen in: /run/media/floris/FLORIS_3/Data_set/PubMed/dataset_large.txt

In [113]:
keyword_list = ["cancer", "melanoma", "carcinoma"] # Niet hoofdlettergevoelig...

trainings_data = xml_parser("/home/floris/Documenten/Dataset/PMC000xxxxxx", "/home/floris/Documenten/Dataset/trainingsData.txt", keyword_list)

# Filter de data:
#trainings_data.run()



## Resultaten:

Het eerste model was met de volgende hyperparameters getrained:

vocabulary grootte: 20000 \
context lengte: 100 \
window_size=5 \
num_ns=4 \
dims = 128

Het model was erg overfit op de trainingsdata, de loss was bijna 0 en accuratesse bijna 1. \
De volgende poging zal een minder grote vocabulary krijgen en minder dimenties om beter te generaliseren i.p.v. de trainingsdata te onthouden. 

De onderstaande afbeelding weergeeft de loss en accuracy grafieken met Tensorboard:

![](img/run_1.png)


Tweede run:

Na het proberen van vele combinaties van hyperparameters lijken deze een veel beter resultaat te geven vergeleken met de vorige:

vocabulary grootte: 1000 \
context lengte: 1000 \
window_size=5 \
num_ns=4 \
dims = 64

![](img/run_2.png)
